In [22]:
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

In [2]:
f = open("biblia-em-txt.txt", "r")
text = f.read()

Dando uma olhada no txt:

In [3]:
print(text[2995:3150])

 domine ele sobre os peixes do mar, sobre as aves do céu, sobre os animais domésticos, e sobre toda a terra, e sobre todo réptil que se arrasta sobre a ter


In [4]:
def tokenize_sentece(sent):
    tokenized = word_tokenize(sent)
    tokenized = [token.lower() for token in tokenized if token.isalpha()]
    return tokenized

def tokenize_text(text):
    sentences = sent_tokenize(text)
    tokenized_senteces = [tokenize_sentece(sent) for sent in sentences]
    return tokenized_senteces

In [5]:
sent_tokens = tokenize_text(text)

Vamos ver os tokens

In [6]:
sent_tokens[200]

['tudo',
 'o',
 'que',
 'tinha',
 'fôlego',
 'do',
 'espírito',
 'de',
 'vida',
 'em',
 'suas',
 'narinas',
 'tudo',
 'o',
 'que',
 'havia',
 'na',
 'terra',
 'seca',
 'morreu']

In [7]:
def make_vocab_and_freq(sent_tokenized):
    i = 0
    vocab = {}
    freqs = {}
    for sent in sent_tokenized:
        for token in sent:
            if token in vocab:
                freqs[token]+=1
            else:
                vocab[token] = i
                freqs[token] = 1
                i+=1
    return vocab,freqs

In [8]:
vocab,freqs = make_vocab_and_freq(sent_tokens)
V = len(vocab)

In [9]:
sorted(vocab)[:15]

['a',
 'aará',
 'aazai',
 'aba',
 'abadom',
 'abafado',
 'abagta',
 'abaixa',
 'abaixam',
 'abaixando',
 'abaixaram',
 'abaixará',
 'abaixarás',
 'abaixava',
 'abaixavam']

In [10]:
vocab['aazai']

13654

Criando as frequências corrigidas das palavras, para o negative sampling

In [11]:
def fixed_distribution(freqs,alpha=3/4):
    noise_dist = {key: val ** alpha for key, val in freqs.items()}
    denominator = sum(noise_dist.values())
    noise_dist_normalized = {key: val / denominator for key, val in noise_dist.items()}
    return noise_dist_normalized

In [12]:
original_dist = np.random.exponential(size = 10000)
original_dist = original_dist/np.sum(original_dist)
trans_original = original_dist**(3/4)
transformed_dist = trans_original/np.sum(trans_original)

In [13]:
freqs_fixed = fixed_distribution(freqs)

In [14]:
np.random.choice(list(freqs_fixed.keys()), p=list(freqs_fixed.values()), size=3, replace = True)

array(['vir', 'tribulação', 'regeneração'], dtype='<U19')

In [15]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

In [16]:
def sigmoid(z):
    """
    Compute sigmoid function
    
    Inputs:
    ----
    z (int or np.array): value on which to perform the sigmoid
    
    Outputs:
    ----
    Value of sigmoid
    
    """
    return 1/(1 + np.exp(-z))

In [17]:
def initialize_params(V, d):
    """
    Initialize params from context and central words
    
    Inputs:
    ----
    V (int): number of words in the vocabulary
    d (int): desired embedding dimention
    
    Outputs:
    ----
    V_W (numpy array): random initialization for words matrix
    V_C (numpy array): random initialization for context matrix
    """
    #central words
    V_W = np.random.randn(V,d)
    #context words
    V_C = np.random.randn(V,d)
    
    return V_W, V_C

In [18]:
def compute_cost(positive_sigmoid, negative_sigmoids):
    """
    Compute cost from training
    
    Inputs:
    ----
    positive_sigmoid (int): σ(v^T_wO.vw_I )
    negative_sigmoids (numpy array): σ(-v^T_wO.vw_I ) para cada negativo
    
    Outputs:
    ----
    E (int): computed cost
    """
    E_1 = -np.log(positive_sigmoid)
    E_2 = -np.sum(np.log(negative_sigmoids))
    E = E_1 + E_2
    return E 

In [19]:
def get_gradients(v_w,v_c, W_neg, positive_sigmoid, negative_sigmoids):
    #gradient central word
    negative_part = [negative_sigmoids[i]*W_neg[i] for i in range(len(W_neg))]
    d_central = (positive_sigmoid - 1)*v_c + np.sum(negative_part)
    #gradient positive word
    d_pos = (positive_sigmoid - 1)*v_w
    #gradient
    d_neg = [n_sig*v_w for n_sig in negative_sigmoids]
    return d_central, d_pos, d_neg

In [23]:
sent_tokens_orig = sent_tokens
vocab = vocab
V = len(vocab)
distribution = freqs_fixed
d=100
C=2
k=5
learning_rate = 0.005
verbose=True
    
V_W, V_C = initialize_params(V, d)
j = -1
costs = []
for i in range(2):
    print(f"EPOCH: {i}")
    for sentence in tqdm(sent_tokens):
        j+=1
        if verbose==True and j%1000==0 and j!=0:
            print("iteração: {:.0f} | custo médio 100 ultimas its: {:.4f}".format(int(j),np.mean(costs[-100:])))
            encoded_embeddings = {}
            for word, i in vocab.items():
                encoded_embeddings[word] = list(V_W[i])
            with open(f'models/bible_embeddings_{j}.json', 'w') as file:
                file.write(json.dumps(encoded_embeddings)) 

        for context_words, central_word in get_windows(sentence,C):
            for context in context_words:
                #negative sampling
                W_neg_words = np.random.choice(list(vocab.keys()), p=list(freqs_fixed.values()), size=k)

                #getting the vectors
                v_w = V_W[vocab[central_word]]
                v_c = V_C[vocab[context]]
                W_neg = np.array([V_C[vocab[w]] for w in W_neg_words])

                #auxiliar value
                positive_sigmoid = sigmoid(np.dot(v_c.T,v_w))
                negative_sigmoids_1 = np.array([sigmoid(np.dot(-w_neg.T,v_w)) for w_neg in W_neg])
                negative_sigmoids = np.array([sigmoid(np.dot(w_neg.T,v_w)) for w_neg in W_neg])

                #get gradients
                d_central, d_pos, d_neg = get_gradients(v_w,v_c, W_neg, positive_sigmoid, negative_sigmoids)

                #calculate cost
                cost = compute_cost(positive_sigmoid, negative_sigmoids_1)
                costs.append(cost)

                #update params 
                V_W[vocab[central_word]]-=learning_rate*d_central
                V_C[vocab[context]]-=learning_rate*d_pos
                for word,g_wj in zip(W_neg_words,d_neg):
                    V_C[vocab[word]] -= learning_rate*g_wj
            

  0%|          | 0/30118 [00:00<?, ?it/s]

EPOCH: 0


  3%|▎         | 1000/30118 [07:24<2:30:16,  3.23it/s]

iteração: 1000 | custo médio 100 ultimas its: 18.6258


  7%|▋         | 2000/30118 [15:46<4:51:34,  1.61it/s] 

...

 96%|█████████▌| 28882/30118 [3:41:26<17:37,  1.17it/s]  

iteração: 59000 | custo médio 100 ultimas its: 7.9360


 99%|█████████▉| 29882/30118 [3:51:42<02:33,  1.53it/s]

iteração: 60000 | custo médio 100 ultimas its: 11.4878


100%|██████████| 30118/30118 [3:53:59<00:00,  2.15it/s]


In [25]:
def cossine_similarity(x,y):
    return np.dot(x.T,y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [26]:
def find_most_similar(x, word_emb):
    similar = {}
    for word, vec in word_emb.items():
        sim = cossine_similarity(np.array(word_emb[x]),np.array(vec))
        if word != x:
            similar[word] = sim
    similar =  {k: v for k, v in sorted(similar.items(), key=lambda item: item[1], reverse=True)}
    most_10_similar = list(similar.keys())[:10]
    most_similar = [(x, similar[x]) for x in most_10_similar]
    return most_similar

In [27]:
find_most_similar('rei', encoded_embeddings)

[('irmãos', 0.9289269710573728),
 ('esta', 0.9259679552027191),
 ('ele', 0.9245393838274762),
 ('uma', 0.923422744078026),
 ('dizendo', 0.9232049396251653),
 ('pois', 0.9227717454692973),
 ('ia', 0.9212027810710568),
 ('ó', 0.9211967719337417),
 ('homem', 0.9211084747274799),
 ('davi', 0.9209030807430225)]

In [28]:
find_most_similar('mulher', encoded_embeddings)

[('habitantes', 0.9317869126492995),
 ('átrio', 0.9316115285762017),
 ('portas', 0.9312491221524574),
 ('sois', 0.9274494632600175),
 ('falou', 0.9272689413311233),
 ('chegado', 0.9272580839422789),
 ('fazendo', 0.9272391510330231),
 ('grandes', 0.9271587598731948),
 ('quer', 0.9258465266561224),
 ('vir', 0.9257120289916091)]

In [29]:
find_most_similar('ir', encoded_embeddings)

[('colunas', 0.9335873065725974),
 ('levitas', 0.9328224342815542),
 ('príncipes', 0.9327844203833894),
 ('lá', 0.932681748183787),
 ('águas', 0.9318841324396119),
 ('cativeiro', 0.9317898737899123),
 ('santuário', 0.9314836620538367),
 ('mandamentos', 0.9308822713976851),
 ('arraial', 0.9304567606155713),
 ('trabalho', 0.9300000858074456)]